# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [21]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36"
}

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}...")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1...
   ---> 100 total reviews
Scraping page 2...
   ---> 200 total reviews
Scraping page 3...
   ---> 300 total reviews
Scraping page 4...
   ---> 400 total reviews
Scraping page 5...
   ---> 500 total reviews
Scraping page 6...
   ---> 600 total reviews
Scraping page 7...
   ---> 700 total reviews
Scraping page 8...
   ---> 800 total reviews
Scraping page 9...
   ---> 900 total reviews
Scraping page 10...
   ---> 1000 total reviews


In [22]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,✅ Trip Verified | Check-in was slow with onl...
1,Not Verified | Check in was smooth enough. Bo...
2,✅ Trip Verified | Although transferring to thi...
3,✅ Trip Verified | We are extremely grateful ...
4,✅ Trip Verified | I had an appalling experie...


In [23]:
# Cleaning the messy data

def clean_text(text):
    text = text.lower() # convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text) # keep only letters and spaces
    text = re.sub(r'sub\s+', '', text) # replace multiple spaces with one 
    text = text.strip() # remove leading/trailing spaces
    text = re.sub(r'✅?\s*Trip Verified\s*\|?', '', text, flags=re.IGNORECASE) # replace ✅ trip verified
    text = re.sub(r'not verified\s*\|?', '', text, flags=re.IGNORECASE) # remove 'not verified
    return text

In [24]:
#Applying the cleaning function to the DataFrame
df['cleaned_reviews'] = df['reviews'].apply(clean_text)
df.head()

,reviews,cleaned_reviews
0,✅ Trip Verified | Check-in was slow with onl...,checkin was slow with only one staff member ch...
1,Not Verified | Check in was smooth enough. Bo...,check in was smooth enough boarding was chaoti...
2,✅ Trip Verified | Although transferring to thi...,although transferring to this flight through h...
3,✅ Trip Verified | We are extremely grateful ...,we are extremely grateful to the crew of fligh...
4,✅ Trip Verified | I had an appalling experie...,i had an appalling experience with british air...


In [34]:
import os

#create the 'data' directory if it doesnt exist
os.makedirs('data', exist_ok=True)

df.to_csv("data/BA_reviews.csv", index=False)

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.